In [10]:
import pandas as pd
import numpy as np
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import Location 
from basketball_reference_web_scraper.data import Team
from basketball_reference_web_scraper.data import Outcome 
from basketball_reference_web_scraper.data import OutputType  
from basketball_reference_web_scraper.data import OutputWriteOption 
from basketball_reference_web_scraper.data import Position
import time
from datetime import datetime, timedelta
from colorama import Fore, Style

In [74]:
#allows me to gather players info
def player(name,year):
    data = pd.DataFrame(client.regular_season_player_box_scores(
        player_identifier=name_dict[name], 
        season_end_year=year
    ))
    data['total_rebounds'] = data['offensive_rebounds'] + data['defensive_rebounds']
    return data

def advance_stats(name,year):
    df = pd.DataFrame(client.players_advanced_season_totals(season_end_year=year))
    return df[name_dict[name]]

#used the get the average of a player for a certain category
def average_stat_for_season(name, year):
    name = player(name,year)
    
    if name.empty:
        print(f"No data found for player: {name}")
        return None
    
    print(f"PPG: {name['points_scored'].mean().round(1)} RPG: {name['total_rebounds'].mean().round(1)} APG: {name['assists'].mean().round(1)}")


#used to calculate the amount of times the player has hit the under or over  on the projected value
def count_projected(player_name, year, stat, projected_value,bet='over'):
    player_data = player(player_name,year)

    proj_value = np.ceil(projected_value) if bet == 'over' else np.floor(projected_value)
    
    if stat not in player_data.columns:
        print(f"Stat '{stat}' not found in the data. ")
        return None

    if player_data.empty:
        print(f"No data found for player: {player_name}")
        return None
    if bet == 'over':    
        count = (player_data[stat] >= proj_value).sum()
    else:
        count = (player_data[stat] <= proj_value).sum()

    games_played = player_data.shape[0]
    percentage = (count/games_played).round(2)

    print(f"{name_dict[player_name]} has gone {bet} on {projected_value} in {columns_rename[stat]} a total of {count} times in {games_played} games played which is {percentage}%.")

#used to calculate the amount of times they've hit the under in most recent games
def single_categories_for_x_games(player_name, stat, projected_value, num_games=None, bet='over'):
    player_data = get_players_info(name_dict[player_name], 2024)

    if num_games is not None and num_games < len(player_data):
        player_data = player_data.tail(num_games)

    proj_value = np.ceil(projected_value) if bet =='over' else np.floor(projected_value)
    
    if stat not in player_data.columns:
        print(f"Stat '{stat}' not found in the data. ")
        return None

    if player_data.empty:
        print(f"No data found for player: {player_name}")
        return None
    
    if bet == 'under':
        count = (player_data[stat] <= proj_value).sum() 
    elif bet == 'over':
        count = (player_data[stat] >= proj_value).sum()

    games_played = player_data.shape[0]
    percentage = (count/num_games).round(2)

    print(f"{player_name} has gone {bet} on {projected_value} in {columns_rename[stat]} a total of {count} times in his last {num_games} games played which is {percentage}%.") 

#calculates the total for rebounds + assists + points
def pts_reb_asts(player_name,projected_value,num_games=None, bet='over'):
    player_data = get_players_info(player_name,2024)

    time.sleep(5)
    
    if num_games is not None and num_games < len(player_data):
        player_data = player_data.tail(num_games)
    
    proj_value = np.ceil(projected_value) if bet == 'over' else np.floor(projected_value)

    if player_data.empty:
        print(f"No data found for player {player_name}")
        return None
    
    total = player_data['points_scored'] + player_data['assists'] + player_data['total_rebounds']
    
    if bet == 'under':
        count = (total <= proj_value).sum()
    else:
        count = (total >= proj_value).sum()
    
    gp = player_data.shape[0]
    percentage = (count/num_games).round(2)

    print(f"{player_name} has gone {bet} on {projected_value} in points + assists + rebounds a total of {count} times in his last {num_games} games played which is {percentage}%.") 

#calculates the total for rebounds + points
def pts_reb(player_name,projected_value,num_games=None, bet='over'):
    player_data = get_players_info(player_name,2024)

    time.sleep(5)
    
    if num_games is not None and num_games < len(player_data):
        player_data = player_data.tail(num_games)
    
    proj_value = np.ceil(projected_value) if bet == 'over' else np.floor(projected_value)

    if player_data.empty:
        print(f"No data found for player {player_name}")
        return None
    
    total = player_data['points_scored'] + player_data['total_rebounds']
    
    if bet == 'under':
        count = (total <= proj_value).sum()
    else:
        count = (total >= proj_value).sum()

    gp = player_data.shape[0]
    percentage = (count/num_games).round(2)

    print(f"{player_name} has gone {bet} on {projected_value} in points + rebounds a total of {count} times in his last {num_games} games played which is {percentage}%.") 

#calculates the total for assists + points
def pts_ast(player_name,projected_value,num_games=None, bet='over'):
    player_data = get_players_info(player_name,2024)

    time.sleep(5)
    
    if num_games is not None and num_games < len(player_data):
        player_data = player_data.tail(num_games)
    
    proj_value = np.ceil(projected_value) if bet == 'over' else np.floor(projected_value)

    if player_data.empty:
        print(f"No data found for player {player_name}")
        return None
    
    total = player_data['points_scored'] + player_data['assists']
    
    if bet == 'under':
        count = (total <= proj_value).sum()
    else:
        count = (total >= proj_value).sum()

    gp = player_data.shape[0]
    percentage = (count/num_games).round(2)

    print(f"{player_name} has gone {bet} on {projected_value} in points + assists a total of {count} times in his last {num_games} games played which is {percentage}%.") 


def reb_ast(player_name,projected_value,num_games=None, bet='over'):
    player_data = get_players_info(player_name,2024)

    time.sleep(5)
    
    if num_games is not None and num_games < len(player_data):
        player_data = player_data.tail(num_games)
    
    proj_value = np.ceil(projected_value) if bet == 'over' else np.floor(projected_value)

    if player_data.empty:
        print(f"No data found for player {player_name}")
        return None
    
    total = player_data['total_rebounds'] + player_data['assists']
    
    if bet == 'under':
        count = (total <= proj_value).sum()
    else:
        count = (total >= proj_value).sum()

    gp = player_data.shape[0]
    percentage = (count/num_games).round(2)

    print(f"{player_name} has gone {bet} on {projected_value} in rebounds + assists a total of {count} times in his last {num_games} games played which is {percentage}%.") 

def blks_stls(player_name,projected_value,num_games=None, bet='over'):
    player_data = get_players_info(player_name,2024)

    time.sleep(5)
    
    if num_games is not None and num_games < len(player_data):
        player_data = player_data.tail(num_games)
    
    proj_value = np.ceil(projected_value) if bet == 'over' else np.floor(projected_value)

    if player_data.empty:
        print(f"No data found for player {player_name}")
        return None
    
    total = player_data['blocks'] + player_data['steals']
    
    if bet == 'under':
        count = (total <= proj_value).sum()
    else:
        count = (total >= proj_value).sum()

    gp = player_data.shape[0]
    percentage = (count/num_games).round(2)

    print(f"{player_name} has gone {bet} on {projected_value} in blocks + steals a total of {count} times in his last {num_games} games played which is {percentage}%.") 


In [65]:
#import slug name into a dictionary with name as value 
df = pd.DataFrame(client.players_season_totals(season_end_year=2024))

name_dict = {}

for index,row in df.iterrows():
    slug = row['slug']
    name = row['name'].lower()
    if name not in name_dict:
        if slug not in name_dict.values():
            name_dict[name] = slug
        else:
            print(f"Player slug {slug} already exists in the dictionary with a different name.")
    else:
        print(f"Name {name} already exists in the dictionary with slug {name_dict[name]}")

columns_rename = {
    'games_played': 'games played',
    'made_field_goals': 'made field goals',
    'attempted_field_goals': 'attempted field goals',
    'made_three_point_field_goals': 'made three point field goals',
    'attempted_three_point_field_goals': 'attempted three point field goals',
    'made_free_throws': 'made free throws',
    'attempted_free_throws': 'attempted free throws',
    'offensive_rebounds': 'offensive rebounds',
    'defensive_rebounds': 'defensive rebounds',
    'personal_fouls': 'personal fouls',
    'assists': 'assists',
    'steals':'steals',
    'blocks':'blocks',
    'turnovers':'turnovers',
    'points_scored': 'points',
    'game_score': 'game score',
    'plus_minus': 'plus minus',
    'total_rebounds': 'total rebounds'
    
}

Name precious achiuwa already exists in the dictionary with slug achiupr01
Name ochai agbaji already exists in the dictionary with slug agbajoc01
Name og anunoby already exists in the dictionary with slug anunoog01
Name marvin bagley iii already exists in the dictionary with slug baglema01
Name dalano banton already exists in the dictionary with slug bantoda01
Name rj barrett already exists in the dictionary with slug barrerj01
Name keita bates-diop already exists in the dictionary with slug bateske01
Name nicolas batum already exists in the dictionary with slug batumni01
Name dāvis bertāns already exists in the dictionary with slug bertada01
Name patrick beverley already exists in the dictionary with slug beverpa01
Name bismack biyombo already exists in the dictionary with slug biyombi01
Name bojan bogdanović already exists in the dictionary with slug bogdabo02
Name marques bolden already exists in the dictionary with slug boldema01
Name bruce brown already exists in the dictionary wi

In [66]:
print(f"The size of the dictionary is {len(name_dict)}.")

The size of the dictionary is 560.


In [67]:
single_categories_for_x_games('zach lavine', 'steals', 2.5 , num_games=15, bet='over')

zach lavine has gone over on 2.5 in steals a total of 2 times in his last 15 games played which is 0.13%.


In [68]:
print('''
    What are you looking for? 
    Options: 
    1 - players information
    2 - players current averages for pts/rebs/asts this season
    3 - How many times the player has hit his projected value for a certain category this season
    4 - How many times the player has hit his projected value an in his x last games this season
      ''')

input =('press which num: ')
if input == 1:
    print('what player do you want? ')


    What are you looking for? 
    Options: 
    1 - players information
    2 - players current averages for pts/rebs/asts this season
    3 - How many times the player has hit his projected value for a certain category this season
    4 - How many times the player has hit his projected value an in his x last games this season
      


In [69]:
average_stat_for_season('lebron james',2024)

PPG: 25.4 RPG: 7.2 APG: 8.0


In [55]:
player

,date,team,location,opponent,outcome,active,seconds_played,made_field_goals,attempted_field_goals,made_three_point_field_goals,...,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points_scored,game_score,plus_minus,total_rebounds
0,2023-10-24,Team.LOS_ANGELES_LAKERS,Location.AWAY,Team.DENVER_NUGGETS,Outcome.LOSS,True,1740,10,16,1,...,7,5,1,0,0,1,21,20.3,7,8
1,2023-10-26,Team.LOS_ANGELES_LAKERS,Location.HOME,Team.PHOENIX_SUNS,Outcome.WIN,True,2100,7,14,1,...,7,9,2,2,5,1,21,20.3,22,8
2,2023-10-29,Team.LOS_ANGELES_LAKERS,Location.AWAY,Team.SACRAMENTO_KINGS,Outcome.LOSS,True,2345,11,19,3,...,15,8,0,0,8,2,27,17.8,-5,15
3,2023-10-30,Team.LOS_ANGELES_LAKERS,Location.HOME,Team.ORLANDO_MAGIC,Outcome.WIN,True,1967,7,17,2,...,3,4,3,1,5,0,19,11.9,5,3
4,2023-11-01,Team.LOS_ANGELES_LAKERS,Location.HOME,Team.LOS_ANGELES_CLIPPERS,Outcome.WIN,True,2549,13,19,4,...,12,7,1,2,4,3,35,30.6,6,12


In [72]:
df = pd.DataFrame(client.players_advanced_season_totals(season_end_year=2024))

In [73]:
df.loc[df['name'] == 'LeBron James']

,slug,name,positions,age,team,games_played,minutes_played,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,...,usage_percentage,offensive_win_shares,defensive_win_shares,win_shares,win_shares_per_48_minutes,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,is_combined_totals
278,jamesle01,LeBron James,[Position.POWER_FORWARD],39,Team.LOS_ANGELES_LAKERS,61,2144,23.2,0.622,0.297,...,29.3,4.4,2.4,6.8,0.152,5.4,0.9,6.3,4.5,False


In [75]:
advance_stats('lebron james',2024)

KeyError: 'jamesle01'